In [10]:
import cv2
import numpy as np
import time

In [2]:
img = np.zeros([10, 10], np.float32) # 기본이 np.float64 / CV_32F를 쓸 것이라 맞추어줌 

for i in range(2,7):
    img[i,3:(i+2)] = 1

In [3]:
print(img)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [6]:
ux = np.array([[-1, 0, 1]])
uy = np.array([-1, 0, 1]).transpose()
k = cv2.getGaussianKernel(3, 1) # 표준편차 1인 정규분포 그래프 3구간으로 쪼갠거(1차원 가우시안 커널)
g = np.outer(k, k.transpose()) # 2차원 가우시안 필터 생성

dy = cv2.filter2D(img, cv2.CV_32F, uy)
dx = cv2.filter2D(img, cv2.CV_32F, ux)
dyy = dy* dy
dxx = dx * dx
dyx = dy* dy
gdyy = cv2.filter2D(dyy, cv2.CV_32F, g)
gdxx = cv2.filter2D(dxx, cv2.CV_32F, g)
gdyx = cv2.filter2D(dyx, cv2.CV_32F, g)

C = (gdyy * gdxx - gdyx * gdyx) - 0.04*(gdyy + gdxx) * (gdyy + gdxx)


for j in range(1, C.shape[0] - 1):
    for i in range(1, C.shape[1] - 1):
        if C[j, i] > 0.1 and sum(sum(C[j, i] > C[j-1 : j+2, i-1 : i+2])) == 8:
            img[j, i] = 9


np.set_printoptions(precision = 2)


print(dy)
print()
print(dx)
print()
print(dyy)
print()
print(dxx)
print()
print(dyx)
print()
print(gdyy)
print()
print(gdxx)
print()
print(gdyx)
print()
print(C)

print(img)


popping = np.zeros([160, 160], np.uint8)


for j in range(0, 160):
    for i in range(0, 160):
        popping[j, i] = np.uint8((C[j // 16, i // 16] + 0.06) * 700)


cv2.imshow('image display2', popping)
cv2.waitKey()
cv2.destroyAllWindows()

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  9.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0. -8.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0.  0. -1. -1. -1. -1.  0.  0.  0.]
 [ 0.  0.  0. -1. -1. -1. -1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  9.  1. -9. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0. -1. -1.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0. -1. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0. -1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 81.  1.  0.  0.  0.  0.  0.]
 [ 0.  

In [9]:
### SIFT 특징점 추출.

img  = cv2.imread('test_image/mot_color70.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create() # SIFT 검출. DOG(Difference of Gaussian) 사용.
kp, des = sift.detectAndCompute(gray, None) # 특징점(키 포인트), 특성 기술자 찾기

gray = cv2.drawKeypoints(gray, kp, None, flags = cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS) # 위에 찾은거 그리기

cv2.imshow('sift', gray)

k = cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
img1 = cv2.imread('test_image/mot_color70.jpg')[190:350, 440:560] # 버스파트 짜르기
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

img2 = cv2.imread('test_image/mot_color83.jpg')
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)
print('특징점 개수: ', len(kp1), len(kp2))

start = time.time()
flann_matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED) # 매칭, 최근접 이웃
knn_match = flann_matcher.knnMatch(des1, des2, 2) # KNN 적용. k = 2

T = 0.7 # 임계값
good_match = []

for nearest1, nearest2 in knn_match:
    if (nearest1.distance / nearest2.distance) < T: # 최근접 이웃 거리 비율
        good_match.append(nearest1)

print('매칭 걸린 시간: ', time.time() - start)

img_match = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1] + img2.shape[1] , 3), dtype=np.uint8)

cv2.drawMatches(img1, kp1, img2, kp2, good_match, img_match, flags = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS) # 매칭 안된건 안 그림.

cv2.imshow('Good Matches', img_match)

k = cv2.waitKey()
cv2.destroyAllWindows()

특징점 개수:  231 4096
매칭 걸린 시간:  0.06135702133178711


In [19]:
img1 = cv2.imread('test_image/mot_color70.jpg')[190:350, 440:560] # 버스 파트 짜르기
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('test_image/mot_color83.jpg')
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

flann_matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_match = flann_matcher.knnMatch(des1, des2, 2)

T = 0.7
good_match = []

for nearest1, nearest2 in knn_match:
    if (nearest1.distance / nearest2.distance) < T: # 최근접 이웃 거리 비율
        good_match.append(nearest1) 


points1 = np.float32([kp1[gm.queryIdx].pt for gm in good_match]) # kp1(키프인트) 에서 좋은 매칭 이루는거 다 모아서 numpy 배열로 변환
points2 = np.float32([kp2[gm.trainIdx].pt for gm in good_match]) # ''

H, _ = cv2.findHomography(points1, points2, cv2.RANSAC) # RANSAC 호모그래피 적용

h1, w1 = img1.shape[0], img1.shape[1]
h2, w2 = img2.shape[0], img2.shape[1]

box1 = np.float32([[0,0], [0, h1-1], [w1-1, h1-1], [w1 - 1, 0]]).reshape(4, 1, 2) # 이런식으로 2차원 배열 이었던 걸
box2 = cv2.perspectiveTransform(box1, H) # perspectiveTransform으로 3차원 배열로 변환

img2 = cv2.polylines(img2, [np.int32(box2)], True, (0,255,0), 8) # 이미지에서의 녹색선

img_match = np.empty((max(h1, h2), w1 + w2 , 3), dtype = np.uint8) # height, width 값 지정

cv2.drawMatches(img1, kp1, img2, kp2, good_match, img_match, flags= cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('Matches and Homography', img_match)

cv2.waitKey()
cv2.destroyAllWindows()